In [1]:
from PyQt5.QtWidgets import *   # PyQt5 위젯
import cv2 as cv                # OpenCV
import numpy as np              # Numpy
import winsound                 # Windows beep sound
import sys                      # 시스템 종료용

# [1] 파노라마 GUI 메인 클래스
class Panorama(QMainWindow):
    def __init__(self):
        super().__init__()  # QMainWindow 초기화
        self.setWindowTitle('파노라마 영상')     # 창 제목
        self.setGeometry(200, 200, 700, 200)     # 창 위치(x,y) + 크기(width,height)

        # [2] 버튼 5개 + 라벨 생성
        collectButton = QPushButton('영상 수집', self)
        self.showButton = QPushButton('영상 보기', self)
        self.stitchButton = QPushButton('봉합', self)
        self.saveButton = QPushButton('저장', self)
        quitButton = QPushButton('나가기', self)
        self.label = QLabel('환영합니다!', self)

        # [3] 버튼 & 라벨 위치 설정
        collectButton.setGeometry(10, 25, 100, 30)
        self.showButton.setGeometry(110, 25, 100, 30)
        self.stitchButton.setGeometry(210, 25, 100, 30)
        self.saveButton.setGeometry(310, 25, 100, 30)
        quitButton.setGeometry(450, 25, 100, 30)
        self.label.setGeometry(10, 70, 600, 170)

        # [4] 수집, 봉합, 저장 버튼은 처음엔 비활성화
        self.showButton.setEnabled(False)
        self.stitchButton.setEnabled(False)
        self.saveButton.setEnabled(False)

        # [5] 버튼 클릭 시 함수 연결 (Signal-Slot)
        collectButton.clicked.connect(self.collectFunction)
        self.showButton.clicked.connect(self.showFunction)
        self.stitchButton.clicked.connect(self.stitchFunction)
        self.saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

    # [6] 영상 수집 함수
    def collectFunction(self):
        # 수집 전 버튼 다시 초기화
        self.showButton.setEnabled(False)
        self.stitchButton.setEnabled(False)
        self.saveButton.setEnabled(False)

        # 사용자에게 안내
        self.label.setText('c를 여러 번 눌러 수집하고 끝나면 q를 눌러 비디오를 끕니다.')

        # 웹캠 열기 (DirectShow 모드)
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():
            sys.exit('카메라 연결 실패')

        self.imgs = []  # 영상 저장 리스트

        # 비디오 루프
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            cv.imshow('video display', frame)  # 현재 프레임 표시

            key = cv.waitKey(1)
            if key == ord('c'):
                self.imgs.append(frame)  # c 누르면 현재 프레임 저장
                print(self.imgs)
            elif key == ord('q'):
                self.cap.release()
                cv.destroyWindow('video display')
                break

        # 2장 이상이면 나머지 버튼 활성화
        if len(self.imgs) >= 2:
            self.showButton.setEnabled(True)
            self.stitchButton.setEnabled(True)
            self.saveButton.setEnabled(True)

    # [7] 수집된 영상 보기
    def showFunction(self):
        self.label.setText('수집된 영상은 ' + str(len(self.imgs)) + '장 입니다.')

        # 첫 번째 영상은 스택 시작점 (크기 축소)
        stack = cv.resize(self.imgs[0], dsize=(0, 0), fx=0.25, fy=0.25)

        # 나머지 영상 가로로 이어붙이기
        for i in range(1, len(self.imgs)):
            resized = cv.resize(self.imgs[i], dsize=(0, 0), fx=0.25, fy=0.25)
            stack = np.hstack((stack, resized))

        cv.imshow('Image collection', stack)  # 이어붙인 결과 표시

    # [8] 영상 Stitching 함수 (파노라마 봉합)
    def stitchFunction(self):
        stitcher = cv.Stitcher_create()  # OpenCV 파노라마 스티쳐 객체 생성
        status, self.img_stitched = stitcher.stitch(self.imgs)  # 이미지 리스트 봉합 시도

        if status == cv.STITCHER_OK:
            # 성공: 파노라마 출력
            cv.imshow('Image stitched panorama', self.img_stitched)
        else:
            # 실패: 경고음 + 라벨 안내
            winsound.Beep(3000, 500)
            self.label.setText('파노라마 제작에 실패했습니다. 다시 시도하세요.')

    # [9] 스티칭 결과 저장
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        cv.imwrite(fname[0], self.img_stitched)

    # [10] 종료 함수
    def quitFunction(self):
        self.cap.release()  # 카메라 닫기
        cv.destroyAllWindows()  # 모든 OpenCV 창 닫기
        self.close()  # PyQt 윈도우 닫기

# [11] PyQt5 앱 실행 루프
app = QApplication(sys.argv)
win = Panorama()
win.show()
app.exec_()

0

| 단계                | 설명                                      |
| ----------------- | --------------------------------------- |
| **영상 수집**         | 연속된 이미지가 필요함. 시야각이 겹치도록 촬영해야 안정적.       |
| **특징점 추출 & 매칭**   | 이미지 간 겹치는 영역을 찾기 위해 특징점(코너 등)을 추출하고 매칭. |
| **Homography 추정** | 두 이미지 간의 투시 변환 관계를 찾아 이미지들을 맞춤.         |
| **Blending**      | 경계선이 자연스럽도록 색상/밝기 조정.                   |
| **Stitcher 클래스**  | OpenCV에서 위의 단계들을 한 번에 처리하는 편리한 함수 제공.   |